In [26]:
import torch
import os
import numpy as np
import matplotlib.pyplot as plt

from models.model import get_model
from scripts.attribution_methods import generate_attributions
from captum.attr import visualization as viz
from scripts.normalize import normalize
import scripts.datasets as datasets
from scripts.ensemble import generate_ensembles

import warnings
warnings.filterwarnings("ignore")

In [27]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
print(device)

cuda:0


## Parameters

In [28]:
params = {
    "model": "Resnet18_cifar10",
    "dataset": "cifar10",
    "batch_size": 20,
    "max_nr_batches": 50,  # -1 for no early stopping
    "attribution_methods": [
        "gradientshap",
        "deeplift",
        "lime",
        "saliency",
        "smoothgrad",
        "integrated_gradients",
        "guidedbackprop",
        "gray_image",
    ],
    "ensemble_methods": [
        "mean",
        "variance",
        "rbm",
        "flipped_rbm",
        "rbm_flip_detection",
    ],
    "attribution_processing": "filtering",
    "normalization": "min_max",
    "scoring_methods": ["insert", "delete", "irof"],
    "scores_batch_size": 40,
    "package_size": 1,
    "irof_segments": 60,
    "irof_sigma": 4,
    "batches_to_plot": [],
}

In [29]:
attribution_params = {
    "lime": {
        "use_slic": True,
        "n_slic_segments": 100,
    },
    "integrated_gradients": {
        "baseline": "black",
    },
    "noise_normal": {},
    "deeplift": {},
    "gradientshap": {},
    "saliency": {},
    "occlusion": {},
    "smoothgrad": {},
    "guidedbackprop": {},
    "gray_image": {},
}

In [30]:
rbm_params = {
    "batch_size": 15,
    "learning_rate": 0.001,
    "n_iter": 300,
}

## Helper functions

In [31]:
def plot(images, raw_images, attributions, ensemble_attributions, plot_text=""):
    for idx in range(len(images)):
        # idx = 0  # first image of the batch
        orig_image = raw_images[idx].detach().cpu().numpy()
        orig_image = orig_image.transpose(1, 2, 0)
        # For MNIST remove the color dimension
        if orig_image.shape[2] == 1:
            orig_image = orig_image.reshape(orig_image.shape[0:2])
        images = [orig_image]

        # one image for every attribution method
        for j, title in enumerate(params["attribution_methods"]):
            # Remove randoms step 1
            if "noise" in title:
                continue
            attribution_img = attributions[j][idx].cpu().detach().numpy()
            images.append(attribution_img)
            #     # one image for every ensemble method
        for j in range(len(params["ensemble_methods"])):
            ensemble_img = ensemble_attributions[j][idx].cpu().detach().numpy()
            images.append(ensemble_img)

        # Remove the randoms step 2
        non_random = np.array(["noise" not in t for t in params["attribution_methods"]])
        attr_methods = np.array(params["attribution_methods"])[non_random]


In [32]:
def my_plot(images, titles, main_title=""):
    # make a square
    x = int(np.ceil(np.sqrt(len(images))))
    fig, axs = plt.subplots(x, x, figsize=(30, 40))
    fig.suptitle(main_title)

    # Remove the NaNs
    for i in range(len(images)):
        images[i][np.isnan(images[i])] = 0

    # Ensure that all attributions get equal weight during plotting
    mean_max_value = np.mean([np.max(img / np.sum(img)) for img in images[1:]])

    # plot the images
    for i, ax in enumerate(axs.flatten()):
        if i < len(images):
            if i == 0:
                # Show the original image
                ax.imshow(images[i])
            else:
                # Plot the attributions and ensure equal plotting
                img = images[i] / np.sum(images[i]) / mean_max_value
                ax.imshow(img, vmin=0, vmax=1 / 3, cmap="Greens")
            ax.set_axis_off()
            ax.set_title(titles[i])
        else:
            ax.set_visible(False)

    plt.tight_layout()
    plt.show()


## running the code

In [34]:
# classification model
model = get_model(params["model"], device=device)

In [35]:
# dataset and which images to explain the classification for
dataset = datasets.get_dataset(params["dataset"])
dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=1, shuffle=False, num_workers=2
)

img, label = next(iter(dataloader))
img = img.to(device)
label = label.to(device)

Files already downloaded and verified


In [36]:
###########################
#      attributions       #
###########################
attributions = generate_attributions(
    img,
    label,
    model,
    params,
    attribution_params,
    device,
)

zero = torch.Tensor([0]).to(device)
# Set negative values to zero
attributions = torch.max(attributions, zero)
# Make sure we have values in range [0,1]
attributions = normalize(params["normalization"], arr=attributions)

In [37]:
###########################
#        ensembles        #
###########################

ensemble_attributions = generate_ensembles(
    attributions, params["ensemble_methods"], rbm_params, device
)

# make sure it sums to 1
ensemble_attributions = normalize(
    params["normalization"], arr=ensemble_attributions
)
